# Load Amazon dataset
1. Load the dataset consisting of baby product reviews on Amazon.com. Store the data in a data frame products. In SFrame, you would run

In [0]:
import pandas as pd
import numpy as np

In [2]:
# mount colab on your google drive
from google.colab import drive
drive.mount('/content/drive/')
import os

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
os.chdir("/content/drive/My Drive/ML-UW/3-Classification/Week1/Assignment")

In [0]:
products = pd.read_csv('amazon_baby.csv')

# Perform text cleaning
2.We start by removing punctuation, so that words "cake." and "cake!" are counted as the same word.

- Write a function **remove_punctuation** that strips punctuation from a line of text
- Apply this function to every element in the **review** column of **products**, and save the result to a new column **review_clean**.

Refer to your tool's manual for string processing capabilities. Python lets us express the operation in a succinct way, as follows:

**Note: the instructure provided by the course goes wrong， I use an alternate function which required the data type is string. So before the removing punctuation process, I should set the date type first **

In [0]:
# set data type
products.review = products.review.astype(str)

In [6]:
def remove_punctuation(text):
    import string
    return text.translate(str.maketrans('','',string.punctuation))

print(remove_punctuation('I am Niu. How are you?'))

products['review_clean'] = products['review'].apply(remove_punctuation)

I am Niu How are you


**Aside**. In this notebook, we remove all punctuation for the sake of simplicity. A smarter approach to punctuation would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See this page for an example of smart handling of punctuation.

**IMPORTANT**. Make sure to fill n/a values in the **review** column with empty strings (if applicable). The n/a values indicate empty reviews. For instance, Pandas's the fillna() method lets you replace all N/A's in the **review** columns as follows:

In [0]:
products = products.fillna({'review':''})  # fill in N/A's in the review column
products = products.fillna({'review_clean':''})  # fill in N/A's in the review_clean column

In [8]:
# have a glance at your data
products.iloc[1]

name                                        Planetwise Wipe Pouch
review          it came early and was not disappointed. i love...
rating                                                          5
review_clean    it came early and was not disappointed i love ...
Name: 1, dtype: object

# Extract Sentiments
3.We will **ignore** all reviews with rating = 3, since they tend to have a neutral sentiment. In SFrame, for instance,

In [0]:
products = products[products['rating'] != 3]

4.Now, we will assign reviews with a rating of 4 or higher to be positive reviews, while the ones with rating of 2 or lower are negative. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label. A good way is to create an anonymous function that converts a rating into a class label and then apply that function to every element in the **rating** column. In SFrame, you would use apply():

In [0]:
products['sentiment'] = products['rating'].apply(lambda rating : +1 if rating > 3 else -1)

Now, we can see that the dataset contains an extra column called sentiment which is either positive (+1) or negative (-1).

In [11]:
products.head()

,name,review,rating,review_clean,sentiment
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed i love ...,1
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...,1
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5,This is a product well worth the purchase I h...,1
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,All of my kids have cried nonstop when I tried...,1
5,Stop Pacifier Sucking without tears with Thumb...,"When the Binky Fairy came to our house, we did...",5,When the Binky Fairy came to our house we didn...,1


# Split into training and test sets
5.Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. If you are using SFrame, make sure to use seed=1 so that you get the same result as everyone else does. (This way, you will get the right numbers for the quiz.)

**Note: the instructure provided by the course goes wrong and you can't use the API by sklearn in order to get the correct answer respect to the course.**

In [12]:
train_indice = open('module-2-assignment-train-idx.json','r')
train_indice

<_io.TextIOWrapper name='module-2-assignment-train-idx.json' mode='r' encoding='UTF-8'>

In [0]:
train_list = []
i = 0
for line in train_indice:
    train_list = [int(x.strip('[]')) for x in line.split(',')]

In [0]:
train_data = products.iloc[train_list]

In [0]:
test_indice = open('module-2-assignment-test-idx.json','r')

In [0]:
test_list = []
i = 0
for line in test_indice:
    test_list = [int(x.strip('[]')) for x in line.split(',')]

In [0]:
test_data = products.iloc[test_list]

# Build the word count vector for each review
6.We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as **bag-of-word features**. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

- Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.
- Compute the occurrences of the words in each review and collect them into a row vector.
- Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix **train_matrix**.
- Using the same mapping between words and columns, convert the test data into a sparse matrix **test_matrix**.

The following cell uses CountVectorizer in scikit-learn. Notice the **token_pattern** argument in the constructor.



In [0]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['review_clean'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['review_clean'])

**Note: Keep in mind that the test data must be transformed in the same way as the training data.**

Because we should make sure the features in both set have the same meaning.

# Train a sentiment classifier with logistic regression
We will now use logistic regression to create a sentiment classifier on the training data.

7.Learn a logistic regression classifier using the training data. If you are using scikit-learn, you should create an instance of the LogisticRegression class and then call the method fit() to train the classifier. This model should use the sparse word count matrix (train_matrix) as features and the column sentiment of train_data as the target. Use the default values for other parameters. Call this model sentiment_model.


In [19]:
# explore the data
print(type(train_matrix))
print(train_matrix.shape)
print(train_data.info())

<class 'scipy.sparse.csr.csr_matrix'>
(133416, 121713)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 133416 entries, 1 to 183529
Data columns (total 5 columns):
name            133174 non-null object
review          133416 non-null object
rating          133416 non-null int64
review_clean    133416 non-null object
sentiment       133416 non-null int64
dtypes: int64(2), object(3)
memory usage: 6.1+ MB
None


In [20]:
# Build a basic model
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.preprocessing import StandardScaler

clf_l1_LR = LogisticRegression(C=0.01, penalty='l1', tol=0.01, solver='saga')
clf_l1_LR.fit(train_matrix,train_data['sentiment'])

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l1', random_state=None, solver='saga',
          tol=0.01, verbose=0, warm_start=False)

In [21]:
from sklearn.linear_model import LogisticRegression
sentiment_model = LogisticRegression()
sentiment_model.fit(train_matrix,train_data['sentiment'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

8.There should be over 100,000 coefficients in this sentiment_model. Recall from the lecture that positive weights w_j correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. Calculate the number of positive (>= 0, which is actually nonnegative) coefficients.

**Quiz question**: How many weights are >= 0?

In [22]:
# Note： you can't assign a object to W directly, the operation in W will change the trained parameters
# W = sentiment_model.coef_
W = np.copy(sentiment_model.coef_)
print(type(W))
print(W.size)

W[W>=0] = 1
W[W<0] = 0
print(np.sum(W))

<class 'numpy.ndarray'>
121713
86999.0


# Making predictions with logistic regression
9.Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 data points in the test data. Take the 11th, 12th, and 13th data points in the test data and save them to **sample_test_data**. The following cell extracts the three data points from the SFrame **test_data** and print their content:

In [23]:
sample_test_data = test_data[10:13]
print(sample_test_data)

                                                 name  \
59                          Our Baby Girl Memory Book   
71  Wall Decor Removable Decal Sticker - Colorful ...   
91  New Style Trailing Cherry Blossom Tree Decal R...   

                                               review  rating  \
59  Absolutely love it and all of the Scripture in...       5   
71  Would not purchase again or recommend. The dec...       2   
91  Was so excited to get this product for my baby...       1   

                                         review_clean  sentiment  
59  Absolutely love it and all of the Scripture in...          1  
71  Would not purchase again or recommend The deca...         -1  
91  Was so excited to get this product for my baby...         -1  


Let's dig deeper into the first row of the** sample_test_data**. Here's the full review:



In [24]:
sample_test_data.iloc[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the rating (-1), the review is quite negative.

In [25]:
sample_test_data.iloc[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

10. We will now make a class prediction for the **sample_test_data**. The **sentiment_model ** should predict +1 if the sentiment is positive and -1 if the sentiment is negative. Recall from the lecture that the score (sometimes called margin) for the logistic regression model is defined as:

scorei=w⊺h(xi)
where h(\mathbf{x}_i)h(x 
i
​	 ) represents the features for data point i. We will write some code to obtain the scores. For each row, the score (or margin) is a number in the range (-inf, inf). Use a pre-built function in your tool to calculate the score of each data point in **sample_test_data**. In scikit-learn, you can call the decision_function() function.

**Hint**: You'd probably need to convert sample_test_data into the sparse matrix format first.

In [26]:
sample_test_matrix = vectorizer.transform(sample_test_data['review_clean'])
scores = sentiment_model.decision_function(sample_test_matrix)
print(scores)

[  5.58730659  -3.18375678 -10.42965697]


# Prediciting Sentiment

11. These scores can be used to make class predictions as follows:

In [27]:
label = np.copy(scores)
label[label>0] = 1
label[label<=0] = -1
label

array([ 1., -1., -1.])

In [28]:
# chechkpoint
sentiment_model.predict(sample_test_matrix) == label

array([ True,  True,  True])

# Probability Predictions
12. Recall from the lectures that we can also calculate the probability predictions from the scores using:

In [29]:
probablity_label = 1/(1 + np.exp(-scores))
probablity_label

array([9.96268872e-01, 3.97815803e-02, 2.95423269e-05])

# Find the most positive (and negative) review
13. We now turn to examining the full test dataset, **test_data**, and use sklearn.linear_model.LogisticRegression to form predictions on all of the test data points.

Using the sentiment_model, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:

- Make probability predictions on **test_data** using the **sentiment_model**.
- Sort the data according to those predictions and pick the top 20.

**Quiz Question**: Which of the following products are represented in the 20 most positive reviews?

In [0]:
test_label_scores = sentiment_model.decision_function(test_matrix)

In [0]:
test_probablity_label = 1/(1 + np.exp(-test_label_scores))
sorted_index = test_probablity_label.argsort()
most_neg_sorted_index = sorted_index[:20]
most_pos_sorted_index = sorted_index[-20:]

14. Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the 20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review.** Repeat the same steps above but make sure you** sort in the opposite order**.

**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?



In [32]:
test_data.iloc[most_neg_sorted_index]

,name,review,rating,review_clean,sentiment
16042,Fisher-Price Ocean Wonders Aquarium Bouncer,We have not had ANY luck with Fisher-Price pro...,2,We have not had ANY luck with FisherPrice prod...,-1
120209,Levana Safe N'See Digital Video Baby Monitor w...,This is the first review I have ever written o...,1,This is the first review I have ever written o...,-1
77072,Safety 1st Exchangeable Tip 3 in 1 Thermometer,I thought it sounded great to have different t...,1,I thought it sounded great to have different t...,-1
48694,Adiri BPA Free Natural Nurser Ultimate Bottle ...,I will try to write an objective review of the...,2,I will try to write an objective review of the...,-1
155287,VTech Communications Safe &amp; Sounds Full Co...,"This is my second video monitoring system, the...",1,This is my second video monitoring system the ...,-1
94560,The First Years True Choice P400 Premium Digit...,Note: we never installed batteries in these un...,1,Note we never installed batteries in these uni...,-1
53207,Safety 1st High-Def Digital Monitor,We bought this baby monitor to replace a diffe...,1,We bought this baby monitor to replace a diffe...,-1
81332,Cloth Diaper Sprayer--styles may vary,I bought this sprayer out of desperation durin...,1,I bought this sprayer out of desperation durin...,-1
113995,Motorola Digital Video Baby Monitor with Room ...,DO NOT BUY THIS BABY MONITOR!I purchased this ...,1,DO NOT BUY THIS BABY MONITORI purchased this m...,-1
10677,Philips AVENT Newborn Starter Set,"It's 3am in the morning and needless to say, t...",1,Its 3am in the morning and needless to say thi...,-1


14. Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the 20 reviews in the **test_data** with the** lowest probability **of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

 **Quiz Question**: Which of the following products are represented in the 20 most negative reviews?



In [33]:
test_data.iloc[most_pos_sorted_index]

,name,review,rating,review_clean,sentiment
182089,Summer Infant Wide View Digital Color Video Mo...,I love this baby monitor. I can compare this ...,5,I love this baby monitor I can compare this o...,1
147996,"Baby Jogger City Mini GT Double Stroller, Shad...","We are well pleased with this stroller, and I ...",4,We are well pleased with this stroller and I w...,1
140816,"Diono RadianRXT Convertible Car Seat, Plum",I bought this seat for my tall (38in) and thin...,5,I bought this seat for my tall 38in and thin 2...,1
168081,Buttons Cloth Diaper Cover - One Size - 8 Colo...,"We are big Best Bottoms fans here, but I wante...",4,We are big Best Bottoms fans here but I wanted...,1
137034,Graco Pack 'n Play Element Playard - Flint,My husband and I assembled this Pack n' Play l...,4,My husband and I assembled this Pack n Play la...,1
66059,"Evenflo 6 Pack Classic Glass Bottle, 4-Ounce",It's always fun to write a review on those pro...,5,Its always fun to write a review on those prod...,1
80155,"Simple Wishes Hands-Free Breastpump Bra, Pink,...","I just tried this hands free breastpump bra, a...",5,I just tried this hands free breastpump bra an...,1
119182,Roan Rocco Classic Pram Stroller 2-in-1 with B...,Great Pram Rocco!!!!!!I bought this pram from ...,5,Great Pram RoccoI bought this pram from Europe...,1
180646,Mamas &amp; Papas 2014 Urbo2 Stroller - Black,After much research I purchased an Urbo2. It's...,4,After much research I purchased an Urbo2 Its e...,1
97325,Freemie Hands-Free Concealable Breast Pump Col...,I absolutely love this product. I work as a C...,5,I absolutely love this product I work as a Cu...,1


# Compute accuracy of the classifier
15. We will now evaluate the accuracy of the trained classifier. Recall that the accuracy is given by

 accuracy= # correctly classified examples / # total examples
This can be computed as follows:

 **Step** 1: Use the **sentiment_model** to compute class predictions.

 **Step** 2: Count the number of data points when the predicted class labels match the ground truth labels.

 **Step** 3: Divide the total number of correct predictions by the total number of data points in the dataset.

 **Quiz Question**: What is the accuracy of the **sentiment_model **on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

 **Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

In [36]:
predicted_label = sentiment_model.predict(test_matrix)
accuracy = (np.sum(predicted_label == test_data['sentiment']))/test_data.shape[0]
accuracy

0.9324454043676506

# Learn another classifier with fewer words
16. There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected 20 words to work with. These are:



In [0]:
from sklearn.metrics import accuracy_score
score = accuracy_score(iris.target, pr) = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [55]:
significant_words

['love',
 'great',
 'easy',
 'old',
 'little',
 'perfect',
 'loves',
 'well',
 'able',
 'car',
 'broke',
 'less',
 'even',
 'waste',
 'disappointed',
 'work',
 'product',
 'money',
 'would',
 'return']

Compute a new set of word count vectors using only these words. The CountVectorizer class has a parameter that lets you limit the choice of words when building word count vectors:

In [0]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['review_clean'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['review_clean'])

Compute word count vectors for the training and test data and obtain the sparse matrices **train_matrix_word_subset** and **test_matrix_word_subset**, respectively.


# Train a logistic regression model on a subset of data
17. Now build a logistic regression classifier with **train_matrix_word_subset** as features and **sentiment** as the target. Call this model simple_model.

18. Let us inspect the weights (coefficients) of the **simple_model**. First, build a table to store (word, coefficient) pairs. If you are using SFrame with scikit-learn, you can combine words with coefficients by running


In [39]:
simple_model = LogisticRegression()
simple_model.fit(train_matrix_word_subset,train_data['sentiment'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [40]:
simple_model.coef_

array([[ 1.36368976,  0.94399959,  1.19253827,  0.08551278,  0.52018576,
         1.50981248,  1.67307389,  0.50376046,  0.19090857,  0.05885467,
        -1.65157634, -0.20956286, -0.51137963, -2.03369861, -2.34829822,
        -0.62116877, -0.32055624, -0.89803074, -0.36216674, -2.10933109]])

Sort the data frame by the coefficient value in descending order.

**Note**: Make sure that the intercept term is excluded from this table.

**Quiz Question**: Consider the coefficients of **simple_model.** How many of the 20 coefficients (corresponding to the 20 **significant_words**) are positive for the **simple_model**?

**Quiz Question**: Are the positive words in the **simple_model** also positive words in the **sentiment_model**?

In [41]:
np.sum(simple_model.coef_>0)

10

In [0]:
positive_words = [word for (word, index) in vectorizer.vocabulary_.items() if sentiment_model.coef_[0][index] >= 0]

# Comparing models
19. We will now compare the accuracy of the **sentiment_model** and the **simple_model**.

 First, compute the classification accuracy of the **sentiment_model** on the train_data.

 Now, compute the classification accuracy of the **simple_model** on the train_data.

 **Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?



In [48]:
from sklearn.metrics import accuracy_score
# accuracy of the simple model
score = accuracy_score(train_data['sentiment'], simple_model.predict(train_matrix_word_subset))
score

0.8668225700065959

In [49]:
score = accuracy_score(train_data['sentiment'], sentiment_model.predict(train_matrix))
score

0.9686394435450021

20. Now, we will repeat this exercise on the test_data. Start by computing the classification accuracy of the sentiment_model on the test_data.

Next, compute the classification accuracy of the simple_model on the test_data.

Quiz Question: Which model (sentiment_model or simple_model) has higher accuracy on the TEST set?



In [50]:
score = accuracy_score(test_data['sentiment'], simple_model.predict(test_matrix_word_subset))
score

0.8693604511639069

In [51]:
score = accuracy_score(test_data['sentiment'], sentiment_model.predict(test_matrix))
score

0.9324454043676506

# Baseline: Majority class prediction
21. It is quite common to use the majority class classifier as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

Quiz Question: Enter the accuracy of the majority class classifier model on the test_data. Round your answer to two decimal places (e.g. 0.76).

Quiz Question: Is the sentiment_model definitely better than the majority class classifier (the baseline)?



In [52]:
np.sum(test_data['sentiment']>0)/test_data.shape[0]

0.8427825773938085